# 🚀 Modelo de Clasificación - Odisea Cósmica

**Objetivo**: Predecir si los pasajeros fueron transportados durante la ruptura espacio-tiempo.

**Métrica de Evaluación**: Accuracy (Exactitud)

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import warnings
warnings.filterwarnings('ignore')

# Configurar visualizaciones
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
%matplotlib inline

print('✓ Librerías cargadas exitosamente')


# PASO 1: Análisis Exploratorio de Datos (EDA)

## 1.1 Carga e Inspección Inicial

In [ ]:

df = pd.read_csv('registros_entrenamiento-2.csv')

print('INFORMACIÓN GENERAL DEL DATASET')
print('=' * 70)
print(f'Dimensiones: {df.shape[0]} filas × {df.shape[1]} columnas')
print(f'\nTipos de datos:')
print(df.dtypes)
print(f'\nValores faltantes:')
print(df.isnull().sum())
print(f'\nEstadísticas descriptivas:')
display(df.describe())


## 1.2 Análisis de Valores Faltantes

In [ ]:

print('ANÁLISIS DE VALORES FALTANTES')
print('=' * 70)

# Contar nulos por columna
nulos = df.isnull().sum()
nulos_pct = (df.isnull().sum() / len(df)) * 100

faltantes = pd.DataFrame({
    'Columna': nulos.index,
    'Nulos': nulos.values,
    'Porcentaje': nulos_pct.values
}).sort_values('Nulos', ascending=False)

print(faltantes[faltantes['Nulos'] > 0])

# Visualizar
plt.figure(figsize=(10, 6))
faltantes_viz = faltantes[faltantes['Nulos'] > 0]
plt.barh(faltantes_viz['Columna'], faltantes_viz['Porcentaje'], color='coral')
plt.xlabel('Porcentaje de Valores Faltantes (%)')
plt.title('Análisis de Valores Faltantes por Columna')
plt.tight_layout()
plt.show()


## 1.3 Distribución de la Variable Objetivo (Transportado)

In [ ]:

print('DISTRIBUCIÓN DE TRANSPORTADO')
print('=' * 70)

transportado_counts = df['Transportado'].value_counts()
transportado_pct = df['Transportado'].value_counts(normalize=True) * 100

print(transportado_counts)
print(f'\nPorcentajes:')
print(transportado_pct)

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Gráfico de barras
axes[0].bar(['No Transportado', 'Transportado'], transportado_counts.values, color=['#FF6B6B', '#4ECDC4'])
axes[0].set_ylabel('Cantidad')
axes[0].set_title('Distribución de Transportado (Frecuencia)')
for i, v in enumerate(transportado_counts.values):
    axes[0].text(i, v + 20, str(v), ha='center')

# Gráfico de pastel
axes[1].pie(transportado_counts.values, labels=['No Transportado', 'Transportado'], autopct='%1.1f%%', colors=['#FF6B6B', '#4ECDC4'])
axes[1].set_title('Distribución de Transportado (Proporción)')

plt.tight_layout()
plt.show()


## 1.4 Distribución de Variables Numéricas

In [ ]:

print('ANÁLISIS DE VARIABLES NUMÉRICAS')
print('=' * 70)

# Identificar columnas numéricas
num_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
# Excluir Transportado si está incluida
if 'Transportado' in num_cols:
    num_cols.remove('Transportado')

print(f'Variables numéricas ({len(num_cols)}): {num_cols}')

# Visualizar histogramas
fig, axes = plt.subplots(3, 3, figsize=(15, 12))
axes = axes.flatten()

for idx, col in enumerate(num_cols[:9]):
    axes[idx].hist(df[col].dropna(), bins=30, color='steelblue', edgecolor='black', alpha=0.7)
    axes[idx].set_title(f'{col}')
    axes[idx].set_xlabel('Valor')
    axes[idx].set_ylabel('Frecuencia')

for idx in range(len(num_cols[:9]), 9):
    axes[idx].axis('off')

plt.tight_layout()
plt.show()


## 1.5 Distribución de Variables Categóricas

In [ ]:

print('ANÁLISIS DE VARIABLES CATEGÓRICAS')
print('=' * 70)

# Identificar columnas categóricas
cat_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
print(f'Variables categóricas ({len(cat_cols)}): {cat_cols}')

# Visualizar distribuciones
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.flatten()

for idx, col in enumerate(cat_cols[:6]):
    counts = df[col].value_counts()
    axes[idx].barh(counts.index[:10], counts.values[:10], color='mediumpurple')
    axes[idx].set_title(f'{col} (top 10)')
    axes[idx].set_xlabel('Cantidad')

for idx in range(len(cat_cols[:6]), 6):
    axes[idx].axis('off')

plt.tight_layout()
plt.show()


print('ANÁLISIS DE CORRELACIONES')
print('=' * 70)

# Correlación de variables numéricas
# Convertir Transportado a numérico temporalmente para correlación
df_corr = df.copy()
df_corr['Transportado'] = df_corr['Transportado'].astype(int)
num_cols_all = df_corr.select_dtypes(include=['float64', 'int64']).columns.tolist()
correlaciones = df_corr[num_cols_all].corr()['Transportado'].sort_values(ascending=False)

print('\nTop 10 correlaciones con Transportado:')
print(correlaciones.head(11))  # 11 para excluir Transportado mismo

# Visualizar correlaciones
plt.figure(figsize=(10, 8))
top_corr = correlaciones.head(11)[1:]  # Excluir Transportado
plt.barh(range(len(top_corr)), top_corr.values, color=['green' if x > 0 else 'red' for x in top_corr.values])
plt.yticks(range(len(top_corr)), top_corr.index)
plt.xlabel('Correlación con Transportado')
plt.title('Top 10 Variables Correlacionadas con Transportado')
plt.tight_layout()
plt.show()

# Mapa de calor de correlaciones
plt.figure(figsize=(12, 10))
sns.heatmap(df_corr[num_cols_all].corr(), cmap='coolwarm', center=0, annot=False)
plt.title('Matriz de Correlación - Variables Numéricas')
plt.tight_layout()
plt.show()

In [ ]:

print('ANÁLISIS DE CORRELACIONES')
print('=' * 70)

# Correlación de variables numéricas
num_cols_all = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
correlaciones = df[num_cols_all].corr()['Transportado'].sort_values(ascending=False)

print('\nTop 10 correlaciones con Transportado:')
print(correlaciones.head(11))  # 11 para excluir Transportado mismo

# Visualizar correlaciones
plt.figure(figsize=(10, 8))
top_corr = correlaciones.head(11)[1:]  # Excluir Transportado
plt.barh(range(len(top_corr)), top_corr.values, color=['green' if x > 0 else 'red' for x in top_corr.values])
plt.yticks(range(len(top_corr)), top_corr.index)
plt.xlabel('Correlación con Transportado')
plt.title('Top 10 Variables Correlacionadas con Transportado')
plt.tight_layout()
plt.show()

# Mapa de calor de correlaciones
plt.figure(figsize=(12, 10))
sns.heatmap(df[num_cols_all].corr(), cmap='coolwarm', center=0, annot=False)
plt.title('Matriz de Correlación - Variables Numéricas')
plt.tight_layout()
plt.show()


## 1.7 Hallazgos Clave y Conclusiones

In [ ]:

print('RESUMEN DE HALLAZGOS - PASO 1')
print('=' * 70)

print('''
🔍 INSIGHTS PRINCIPALES:

1. VARIABLE OBJETIVO (Transportado):
   - Distribución relativamente balanceada
   - Aproximadamente 50% en cada clase
   - Bien para modelos de clasificación

2. VALORES FALTANTES:
   - Algunas columnas tienen valores faltantes (revisar EDA anterior)
   - Estrategia: Imputar con media/moda según tipo de variable

3. VARIABLES MÁS PROMETEDORAS:
   - Variables numéricas: Edad, gastos en amenidades (Spa, Cafeteria, etc.)
   - Variables categóricas: PlanetaOrigen, Destino, SuenoCriogenico
   - Estas serán focalizadas en el modelo

4. CORRELACIONES INTERESANTES:
   - Correlaciones bajas a moderadas con Transportado
   - Sugiere relaciones no lineales
   - Random Forest podría ser mejor que Regresión Logística

5. PRÓXIMOS PASOS:
   - Imputar valores faltantes
   - Crear nuevas características si es necesario
   - Codificar variables categóricas
   - Escalar variables numéricas
''')


# PASO 2: Limpieza de Datos e Ingeniería de Características

## 2.1 Análisis Detallado de Valores Faltantes

In [ ]:

print('ANÁLISIS DETALLADO - VALORES FALTANTES')
print('=' * 70)

df_clean = df.copy()

# Agrupar faltantes por tipo de variable
num_cols = df_clean.select_dtypes(include=['float64', 'int64']).columns.tolist()
cat_cols = df_clean.select_dtypes(include=['object', 'category']).columns.tolist()

print('\nVARIABLES NUMÉRICAS CON FALTANTES:')
for col in num_cols:
    if df_clean[col].isnull().sum() > 0:
        print(f'  {col}: {df_clean[col].isnull().sum()} ({df_clean[col].isnull().sum()/len(df_clean)*100:.2f}%)')

print('\nVARIABLES CATEGÓRICAS CON FALTANTES:')
for col in cat_cols:
    if df_clean[col].isnull().sum() > 0:
        print(f'  {col}: {df_clean[col].isnull().sum()} ({df_clean[col].isnull().sum()/len(df_clean)*100:.2f}%)')


## 2.2 Estrategia de Imputación (Justificada)

In [ ]:

print('ESTRATEGIA DE IMPUTACIÓN')
print('=' * 70)

print('''
JUSTIFICACIÓN POR TIPO DE VARIABLE:

📊 VARIABLES NUMÉRICAS:
   - Método: MEDIA
   - Razón: Resistente a outliers moderados, mantiene distribución
   - Alternativas descartadas: Mediana (menos interpretable), KNN (más complejo)

📝 VARIABLES CATEGÓRICAS:
   - Método: MODA (valor más frecuente)
   - Razón: Preserva patrones de la variable, no introduce sesgo
   - Alternativas descartadas: "Desconocido" (sesga modelo), Drop (pierde data)
''')

# Aplicar imputación
from sklearn.impute import SimpleImputer

# Imputar numéricas con media
num_cols = df_clean.select_dtypes(include=['float64', 'int64']).columns.tolist()
if num_cols:
    imputer_num = SimpleImputer(strategy='mean')
    df_clean[num_cols] = imputer_num.fit_transform(df_clean[num_cols])
    print('\n✓ Variables numéricas imputadas con MEDIA')

# Imputar categóricas con moda
cat_cols = df_clean.select_dtypes(include=['object', 'category']).columns.tolist()
if cat_cols:
    imputer_cat = SimpleImputer(strategy='most_frequent')
    df_clean[cat_cols] = imputer_cat.fit_transform(df_clean[cat_cols])
    print('✓ Variables categóricas imputadas con MODA')

# Verificar
print(f'\nValores faltantes después de imputación: {df_clean.isnull().sum().sum()}')


## 2.3 Ingeniería de Características

In [ ]:

print('INGENIERÍA DE CARACTERÍSTICAS')
print('=' * 70)

print('''
NUEVAS CARACTERÍSTICAS CREADAS:

1. TotalGastos: Suma de todos los gastos en amenidades
   Razón: Captura poder adquisitivo total del pasajero
   
2. EsVIP_Binary: Conversión explícita a 0/1
   Razón: Facilita la comparación y análisis

3. EdadGrupo: Categorización de edad en grupos
   Razón: Captura relaciones no lineales con edad

4. EsJoven: Boolean si edad < 18
   Razón: Podría tener relación especial con transporte

5. TieneGastosAltos: Si TotalGastos > percentil 75
   Razón: Divide a pasajeros por nivel de consumo
''')

# Crear características
df_clean['TotalGastos'] = df_clean[['ServicioHabitacion', 'Cafeteria', 'CentroComercial', 'Spa', 'CubiertaVR']].sum(axis=1)

df_clean['EsVIP'] = (df_clean['ServicioVIP'] == True).astype(int)

df_clean['EdadGrupo'] = pd.cut(df_clean['Edad'], bins=[0, 18, 35, 50, 100], labels=['Niño', 'Adulto', 'Mayor', 'Anciano'])

df_clean['EsJoven'] = (df_clean['Edad'] < 18).astype(int)

df_clean['TieneGastosAltos'] = (df_clean['TotalGastos'] > df_clean['TotalGastos'].quantile(0.75)).astype(int)

print('\n✓ 5 nuevas características creadas exitosamente')
print(f'\nDataset ahora tiene {df_clean.shape[1]} columnas')


## 2.4 Eliminación de Columnas sin Valor Predictivo

In [ ]:

print('ELIMINACIÓN DE COLUMNAS')
print('=' * 70)

print('''
COLUMNAS A ELIMINAR Y JUSTIFICACIÓN:

1. IdPasajero: ID único, no tiene información predictiva
2. Iniciales: Identificador personal, sin patrón predictivo
3. Cabina: Demasiado variada, solo ubicación en nave
   
COLUMNAS A MANTENER:
- Todas las demás tienen potencial predictivo (variables, nuevas características)
''')

# Eliminar columnas
columnas_drop = ['IdPasajero', 'Iniciales', 'Cabina']
df_clean = df_clean.drop(columns=columnas_drop)

print(f'\n✓ Columnas eliminadas: {columnas_drop}')
print(f'\nDataset final: {df_clean.shape[0]} filas × {df_clean.shape[1]} columnas')
print(f'Columnas actuales: {df_clean.columns.tolist()}')


## 2.5 Validación Final de Datos Limpios

In [ ]:

print('VALIDACIÓN FINAL - PASO 2')
print('=' * 70)

print(f'\n✓ Sin valores faltantes: {df_clean.isnull().sum().sum() == 0}')
print(f'✓ Shape: {df_clean.shape}')
print(f'✓ Columnas: {len(df_clean.columns)}')
print(f'\nTipos de datos:')
print(df_clean.dtypes)
print(f'\nMuestra de datos limpios:')
display(df_clean.head())


# PASO 3: Pre-procesamiento de Datos para el Modelo

## 3.1 Separar Features (X) y Target (y)

In [ ]:

print('SEPARACIÓN DE FEATURES Y TARGET')
print('=' * 70)

y = df_clean['Transportado'].astype(int)
X = df_clean.drop(columns=['Transportado'])

print(f'Target (y): {y.shape}')
print(f'  - Distribución: {y.value_counts().to_dict()}')
print(f'\nFeatures (X): {X.shape}')
print(f'  - Columnas: {X.columns.tolist()}')


## 3.2 One-Hot Encoding para Variables Categóricas

In [ ]:

print('ONE-HOT ENCODING')
print('=' * 70)

# Identificar categóricas
cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
num_cols = X.select_dtypes(include=['float64', 'int64']).columns.tolist()

print(f'Variables categóricas: {cat_cols}')
print(f'Variables numéricas: {num_cols}')

# One-Hot Encoding
if len(cat_cols) > 0:
    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop='first')
    X_cat_encoded = encoder.fit_transform(X[cat_cols])
    X_cat_encoded = pd.DataFrame(
        X_cat_encoded,
        columns=encoder.get_feature_names_out(cat_cols),
        index=X.index
    )
    print(f'\n✓ One-Hot Encoding completado: {X_cat_encoded.shape}')
    print(f'Nuevas columnas: {X_cat_encoded.columns.tolist()[:10]}...')
else:
    X_cat_encoded = pd.DataFrame(index=X.index)


## 3.3 StandardScaler para Variables Numéricas

In [ ]:

print('STANDARDSCALER')
print('=' * 70)

if len(num_cols) > 0:
    scaler = StandardScaler()
    X_num_scaled = scaler.fit_transform(X[num_cols])
    X_num_scaled = pd.DataFrame(X_num_scaled, columns=num_cols, index=X.index)
    print(f'✓ StandardScaler completado: {X_num_scaled.shape}')
    print(f'\nEjemplo de escalado (primeras 3 filas):')
    display(X_num_scaled.head(3))
else:
    X_num_scaled = pd.DataFrame(index=X.index)


## 3.4 Dataset Final Preprocesado

In [ ]:

print('DATASET FINAL PREPROCESADO')
print('=' * 70)

X_processed = pd.concat([X_cat_encoded, X_num_scaled], axis=1)

print(f'✓ Shape final: {X_processed.shape}')
print(f'✓ Sin valores NaN: {X_processed.isnull().sum().sum() == 0}')
print(f'\nMuestra de datos procesados:')
display(X_processed.head(3))


# PASO 4: Entrenamiento y Evaluación del Modelo

## 4.1 División de Datos (60% Entrenamiento, 20% Validación, 20% Prueba)

In [ ]:

print('DIVISIÓN DE DATOS')
print('=' * 70)

# Primera división: 80% temp, 20% prueba
X_temp, X_test, y_temp, y_test = train_test_split(
    X_processed, y, test_size=0.2, random_state=42, stratify=y
)

# Segunda división: 75% entrenamiento, 25% validación
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp
)

print(f'Entrenamiento: {X_train.shape[0]} ({X_train.shape[0]/len(y)*100:.1f}%)')
print(f'Validación: {X_val.shape[0]} ({X_val.shape[0]/len(y)*100:.1f}%)')
print(f'Prueba: {X_test.shape[0]} ({X_test.shape[0]/len(y)*100:.1f}%)')

print(f'\nDistribución del target - Entrenamiento: {y_train.value_counts().to_dict()}')
print(f'Distribución del target - Validación: {y_val.value_counts().to_dict()}')
print(f'Distribución del target - Prueba: {y_test.value_counts().to_dict()}')


## 4.2 Selección y Justificación de Modelos


### Modelos Seleccionados: Random Forest + Regresión Logística

#### 🌲 Random Forest
**Ventajas:**
- Captura relaciones NO lineales
- Maneja bien datos con correlaciones bajas (como el nuestro)
- Proporciona importancia de características
- Robusto ante valores atípicos

**Desventajas:**
- Menos interpretable que Regresión Logística
- Puede overfitting en datasets pequeños

#### 📊 Regresión Logística  
**Ventajas:**
- Modelo baseline rápido y interpretable
- Probabilidades calibradas
- Excelente como comparación

**Desventajas:**
- Asume relaciones lineales
- Podría subperformar con relaciones complejas

**Decisión:** Entrenar ambos y elegir el que mejor Accuracy tenga en validación.


## 4.3 Entrenamiento de Modelos

In [ ]:

print('ENTRENAMIENTO DE MODELOS')
print('=' * 70)

# Regresión Logística
print('\nEntrenando Regresión Logística...')
import time
start = time.time()
lr_model = LogisticRegression(max_iter=1000, random_state=42, n_jobs=-1)
lr_model.fit(X_train, y_train)
lr_time = time.time() - start
print(f'✓ Completado en {lr_time:.2f}s')

# Random Forest
print('\nEntrenando Random Forest...')
start = time.time()
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1, max_depth=15)
rf_model.fit(X_train, y_train)
rf_time = time.time() - start
print(f'✓ Completado en {rf_time:.2f}s')


## 4.4 Comparación en Conjunto de Validación

In [ ]:

print('EVALUACIÓN EN VALIDACIÓN')
print('=' * 70)

def eval_model(y_true, y_pred, model_name):
    acc = accuracy_score(y_true, y_pred)
    print(f'{model_name} - Accuracy: {acc:.4f}')
    return acc

y_val_pred_lr = lr_model.predict(X_val)
y_val_pred_rf = rf_model.predict(X_val)

acc_lr_val = eval_model(y_val, y_val_pred_lr, 'Regresión Logística')
acc_rf_val = eval_model(y_val, y_val_pred_rf, 'Random Forest')

# Seleccionar mejor modelo
mejor_modelo = rf_model if acc_rf_val > acc_lr_val else lr_model
mejor_nombre = 'Random Forest' if acc_rf_val > acc_lr_val else 'Regresión Logística'
print(f'\n✓ MEJOR MODELO EN VALIDACIÓN: {mejor_nombre}')


## 4.5 Evaluación Final en Conjunto de Prueba

In [ ]:

print('EVALUACIÓN FINAL EN PRUEBA')
print('=' * 70)

y_test_pred = mejor_modelo.predict(X_test)
accuracy_final = accuracy_score(y_test, y_test_pred)

print(f'Modelo: {mejor_nombre}')
print(f'Accuracy: {accuracy_final:.4f} ({accuracy_final*100:.2f}%)')


## 4.6 Matriz de Confusión y Reporte Detallado

In [ ]:

print('MATRIZ DE CONFUSIÓN Y MÉTRICAS')
print('=' * 70)

cm = confusion_matrix(y_test, y_test_pred)
print(f'\nMatriz de Confusión:')
print(cm)

# Visualizar matriz
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['No Transportado', 'Transportado'],
            yticklabels=['No Transportado', 'Transportado'])
plt.title(f'Matriz de Confusión - {mejor_nombre}')
plt.ylabel('Verdadero')
plt.xlabel('Predicción')
plt.tight_layout()
plt.show()

# Reporte de clasificación
print(f'\nReporte Detallado de Clasificación:')
print(classification_report(y_test, y_test_pred, target_names=['No Transportado', 'Transportado']))


## 4.7 Interpretación de Resultados

In [ ]:

print('INTERPRETACIÓN DE RESULTADOS - PASO 4')
print('=' * 70)

tn, fp, fn, tp = cm.ravel()

print(f'''
📊 ANÁLISIS DEL RENDIMIENTO:

✓ Verdaderos Positivos (TP): {tp}
  - Pasajeros correctamente identificados como Transportados

✓ Verdaderos Negativos (TN): {tn}
  - Pasajeros correctamente identificados como No Transportados

⚠ Falsos Positivos (FP): {fp}
  - Pasajeros predichos como Transportados pero no fueron

⚠ Falsos Negativos (FN): {fn}
  - Pasajeros predichos como No Transportados pero sí fueron

📈 MÉTRICAS:
- Accuracy: {accuracy_final:.4f} ({accuracy_final*100:.2f}%)
- Total Errores: {fp + fn}/{len(y_test)} ({(fp+fn)/len(y_test)*100:.2f}%)

🔍 ANÁLISIS DE ERRORES:
''')

if fp > fn:
    print(f'⚠ El modelo SE EQUIVOCA MÁS en Falsos Positivos ({fp} casos)')
    print(f'   → Tiende a predecir "Transportado" demasiado a menudo')
    print(f'   → Recomendación: Ajustar threshold de predicción')
elif fn > fp:
    print(f'⚠ El modelo SE EQUIVOCA MÁS en Falsos Negativos ({fn} casos)')
    print(f'   → Tiende a perderse casos reales de "Transportado"')
    print(f'   → Recomendación: Aumentar sensibilidad del modelo')
else:
    print(f'✓ Errores balanceados entre FP ({fp}) y FN ({fn})')
    print(f'   → Buen equilibrio entre precisión y recall')

# Importancia de características (si aplica)
if hasattr(mejor_modelo, 'feature_importances_'):
    print(f'\n🎯 TOP 10 CARACTERÍSTICAS MÁS IMPORTANTES ({mejor_nombre}):')
    importances = mejor_modelo.feature_importances_
    indices = np.argsort(importances)[-10:][::-1]
    for i, idx in enumerate(indices, 1):
        print(f'  {i:2d}. {X_processed.columns[idx]:30s} → {importances[idx]:.4f}')


# PASO 5: Generación del Archivo de Entrega

## 5.1 Cargar Datos de Predicción

In [ ]:

print('CARGAR DATOS DE PREDICCIÓN')
print('=' * 70)

datos_prediccion = pd.read_csv('registros_evaluacion-2.csv')
print(f'✓ Datos cargados: {datos_prediccion.shape}')
print(f'Primeras filas:')
display(datos_prediccion.head())


## 5.2 Aplicar Limpieza Idéntica al Entrenamiento

In [ ]:

print('LIMPIEZA DE DATOS DE PREDICCIÓN')
print('=' * 70)

df_pred = datos_prediccion.copy()

# Guardar IdPasajero
id_pasajero = df_pred['IdPasajero'].copy()

# Imputar valores faltantes
num_cols_pred = df_pred.select_dtypes(include=['float64', 'int64']).columns.tolist()
cat_cols_pred = df_pred.select_dtypes(include=['object', 'category']).columns.tolist()

if num_cols_pred:
    imputer_num_pred = SimpleImputer(strategy='mean')
    df_pred[num_cols_pred] = imputer_num_pred.fit_transform(df_pred[num_cols_pred])

if cat_cols_pred:
    imputer_cat_pred = SimpleImputer(strategy='most_frequent')
    df_pred[cat_cols_pred] = imputer_cat_pred.fit_transform(df_pred[cat_cols_pred])

# Crear mismas características
df_pred['TotalGastos'] = df_pred[['ServicioHabitacion', 'Cafeteria', 'CentroComercial', 'Spa', 'CubiertaVR']].sum(axis=1)
df_pred['EsVIP'] = (df_pred['ServicioVIP'] == True).astype(int)
df_pred['EdadGrupo'] = pd.cut(df_pred['Edad'], bins=[0, 18, 35, 50, 100], labels=['Niño', 'Adulto', 'Mayor', 'Anciano'])
df_pred['EsJoven'] = (df_pred['Edad'] < 18).astype(int)
df_pred['TieneGastosAltos'] = (df_pred['TotalGastos'] > df_pred['TotalGastos'].quantile(0.75)).astype(int)

# Eliminar mismas columnas
df_pred = df_pred.drop(columns=['IdPasajero', 'Iniciales', 'Cabina'])

print(f'✓ Limpieza completada: {df_pred.shape}')
print(f'✓ Sin valores faltantes: {df_pred.isnull().sum().sum() == 0}')


## 5.3 Aplicar Preprocesamiento Idéntico

In [ ]:

print('PREPROCESAMIENTO DE DATOS DE PREDICCIÓN')
print('=' * 70)

# Separar categóricas y numéricas
cat_cols_pred = df_pred.select_dtypes(include=['object', 'category']).columns.tolist()
num_cols_pred = df_pred.select_dtypes(include=['float64', 'int64']).columns.tolist()

# One-Hot Encoding
X_cat_pred = pd.DataFrame(index=df_pred.index)
if cat_cols_pred:
    encoder_pred = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop='first')
    X_cat_pred = encoder_pred.fit_transform(df_pred[cat_cols_pred])
    X_cat_pred = pd.DataFrame(X_cat_pred, columns=encoder_pred.get_feature_names_out(cat_cols_pred), index=df_pred.index)

# StandardScaler
X_num_pred = pd.DataFrame(index=df_pred.index)
if num_cols_pred:
    scaler_pred = StandardScaler()
    X_num_pred = scaler_pred.fit_transform(df_pred[num_cols_pred])
    X_num_pred = pd.DataFrame(X_num_pred, columns=num_cols_pred, index=df_pred.index)

# Combinar
X_processed_pred = pd.concat([X_cat_pred, X_num_pred], axis=1)

# Ajustar columnas a match del entrenamiento
if X_processed_pred.shape[1] < X_processed.shape[1]:
    missing_cols = set(X_processed.columns) - set(X_processed_pred.columns)
    for col in missing_cols:
        X_processed_pred[col] = 0
    X_processed_pred = X_processed_pred[X_processed.columns]

print(f'✓ Preprocesamiento completado: {X_processed_pred.shape}')
print(f'✓ Coincide con entrenamiento: {X_processed_pred.shape == X_processed.shape}')


## 5.4 Realizar Predicciones

In [ ]:

print('REALIZANDO PREDICCIONES')
print('=' * 70)

y_predicciones = mejor_modelo.predict(X_processed_pred)

print(f'✓ Predicciones realizadas: {len(y_predicciones)}')
print(f'\nDistribución de predicciones:')
unique, counts = np.unique(y_predicciones, return_counts=True)
for u, c in zip(unique, counts):
    label = 'Transportado' if u == 1 else 'No Transportado'
    print(f'  {label}: {c} ({c/len(y_predicciones)*100:.1f}%)')


## 5.5 Generar Archivo de Entrega

In [ ]:

print('GENERANDO ARCHIVO DE ENTREGA')
print('=' * 70)

# Crear DataFrame
entrega = pd.DataFrame({
    'IdPasajero': id_pasajero,
    'Transportado': y_predicciones
})

# Guardar
archivo_salida = 'predicciones_transportado.csv'
entrega.to_csv(archivo_salida, index=False)

print(f'✓ Archivo guardado: {archivo_salida}')
print(f'✓ Dimensiones: {entrega.shape}')
print(f'\nPrimeras 10 filas:')
display(entrega.head(10))
print(f'\nÚltimas 10 filas:')
display(entrega.tail(10))

print(f'\n' + '='*70)
print('✅ GENERACIÓN DE ENTREGA COMPLETADA')
print('='*70)
print(f'Total de predicciones: {len(entrega)}')
print(f'Transportados: {(entrega["Transportado"] == 1).sum()}')
print(f'No Transportados: {(entrega["Transportado"] == 0).sum()}')
print(f'Archivo listo para envío: {archivo_salida}')
